<a href="https://colab.research.google.com/github/beingUday/Stock-market-trends-prediction/blob/main/Predictive_models_and_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install xgboost

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor, StackingRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV,TimeSeriesSplit
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score,mean_squared_log_error,root_mean_squared_error
from sklearn.ensemble import VotingRegressor,StackingRegressor
from sklearn.preprocessing import StandardScaler

In [ ]:
ds1=pd.read_csv("/content/drive/MyDrive/stocks/Scaled.csv")
ds1

,Date,Open,High,Low,Close,Volume,Tomorrow,Stock name,Yfinance ticker,Zerodha token
0,2010-01-04,-0.823037,-0.822634,-0.822972,-0.822650,-0.697873,30.263664,Baja Finance,BAJFINANCE.NS,81153
1,2010-01-04,-0.366181,-0.367472,-0.360584,-0.363668,-0.649173,606.428955,Larsen & Toubro (L&T),LT.NS,2939649
2,2010-01-04,-0.686265,-0.685837,-0.685961,-0.684542,-0.645445,205.466766,Kotak Mahindra Bank,KOTAKBANK.NS,492033
3,2010-01-04,-0.609498,-0.610896,-0.608675,-0.610621,-0.575081,296.118103,Tata Consultancy Services (TCS),TCS.NS,2953217
4,2010-01-04,-0.659808,-0.660541,-0.658831,-0.659807,-0.402440,235.218887,Infosys,INFY.NS,408065
...,...,...,...,...,...,...,...,...,...,...
36975,2024-12-27,1.022123,1.004572,1.032624,1.022248,-0.690432,2343.500000,Hindustan Unilever (HUL),HINDUNILVR.NS,356865
36976,2024-12-27,-0.195539,-0.200938,-0.203294,-0.208545,0.137110,788.299988,State Bank of India (SBI),SBIN.NS,779521
36977,2024-12-27,0.676294,0.671889,0.690073,0.683332,-0.413244,1906.000000,Infosys,INFY.NS,408065
36978,2024-12-27,4.614294,4.659776,4.666916,4.668080,-0.690547,6888.250000,Baja Finance,BAJFINANCE.NS,81153


In [ ]:
Y = ds1["Tomorrow"]
X = ds1.drop(columns={"Date","Stock name","Yfinance ticker","Zerodha token"})
Y

,Tomorrow
0,30.263664
1,606.428955
2,205.466766
3,296.118103
4,235.218887
...,...
36975,2343.500000
36976,788.299988
36977,1906.000000
36978,6888.250000


In [ ]:
X

,Open,High,Low,Close,Volume,Tomorrow
0,-0.823037,-0.822634,-0.822972,-0.822650,-0.697873,30.263664
1,-0.366181,-0.367472,-0.360584,-0.363668,-0.649173,606.428955
2,-0.686265,-0.685837,-0.685961,-0.684542,-0.645445,205.466766
3,-0.609498,-0.610896,-0.608675,-0.610621,-0.575081,296.118103
4,-0.659808,-0.660541,-0.658831,-0.659807,-0.402440,235.218887
...,...,...,...,...,...,...
36975,1.022123,1.004572,1.032624,1.022248,-0.690432,2343.500000
36976,-0.195539,-0.200938,-0.203294,-0.208545,0.137110,788.299988
36977,0.676294,0.671889,0.690073,0.683332,-0.413244,1906.000000
36978,4.614294,4.659776,4.666916,4.668080,-0.690547,6888.250000


In [ ]:
X_train = X.iloc[:25889]
X_test = X.iloc[25890:]

In [ ]:
X_train

,Open,High,Low,Close,Volume
0,-0.823037,-0.822634,-0.822972,-0.822650,-0.697873
1,-0.366181,-0.367472,-0.360584,-0.363668,-0.649173
2,-0.686265,-0.685837,-0.685961,-0.684542,-0.645445
3,-0.609498,-0.610896,-0.608675,-0.610621,-0.575081
4,-0.659808,-0.660541,-0.658831,-0.659807,-0.402440
...,...,...,...,...,...
25884,0.242789,0.234714,0.240871,0.242127,-0.443024
25885,0.773419,0.756362,0.774839,0.758773,-0.570900
25886,-0.566734,-0.558407,-0.564267,-0.556752,3.038068
25887,0.002819,-0.004480,0.000214,0.000049,0.262158


In [ ]:
X_test

,Open,High,Low,Close,Volume
25890,-0.188798,-0.191956,-0.193481,-0.197908,1.412437
25891,1.788363,1.804975,1.683321,1.677427,0.832105
25892,0.757111,0.769996,0.754754,0.782552,-0.461419
25893,0.849911,0.835314,0.823033,0.813029,-0.507054
25894,-0.277203,-0.283332,-0.279853,-0.284406,-0.041071
...,...,...,...,...,...
36975,1.022123,1.004572,1.032624,1.022248,-0.690432
36976,-0.195539,-0.200938,-0.203294,-0.208545,0.137110
36977,0.676294,0.671889,0.690073,0.683332,-0.413244
36978,4.614294,4.659776,4.666916,4.668080,-0.690547


In [ ]:
Y_train = Y.iloc[:25889]
Y_test = Y.iloc[25890:]

In [ ]:
Y_train

,Tomorrow
0,30.263664
1,606.428955
2,205.466766
3,296.118103
4,235.218887
...,...
25884,1348.675293
25885,2041.025635
25886,356.648743
25887,1065.912231


In [ ]:
Y_test

,Tomorrow
25890,824.841980
25891,3282.073975
25892,2031.502075
25893,2065.565186
25894,710.999268
...,...
36975,2343.500000
36976,788.299988
36977,1906.000000
36978,6888.250000


In [ ]:
tscv = TimeSeriesSplit(n_splits=5)

In [ ]:
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

((25889, 5), (11090, 5), (25889,), (11090,))

In [ ]:
svr_pipeline = Pipeline([("scaler", StandardScaler()),("SVR",SVR())])
rfr_pipeline = Pipeline([("scaler", StandardScaler()),("RFR",RandomForestRegressor(random_state=42))])
gbr_pipeline = Pipeline([("scaler", StandardScaler()),("GBR",GradientBoostingRegressor(random_state=42))])
xgb_pipeline = Pipeline([("scaler", StandardScaler()),("XGBR",XGBRegressor(random_state=42))])

svrparam_grid={
        "SVR__kernel": ['rbf'],
        "SVR__gamma": ["scale", "auto"],
        "SVR__C": [0.5, 1, 5],
        "SVR__epsilon": [0.01, 0.05, 0.1]
    }
rfrparam_grid={
        'RFR__n_estimators': [i for i in range(100,501,100)],
        'RFR__max_depth': [5, 10, 15, None],
        'RFR__min_samples_split': [2, 5, 10],
        'RFR__min_samples_leaf': [i for i in range(3,10)]
    }
gbrparam_grid={
        'GBR__n_estimators': [i for i in range(100,501,100)],
        'GBR__learning_rate': [0.01, 0.05, 0.1,0.5],
        'GBR__max_depth': [3, 4, 5, 10],
        'GBR__subsample': [0.8, 1.0],
        "GBR__criterion":["friedman_mse","squared_error"]
    }
xgbparam_grid={
        'XGBR__n_estimators': [100, 200, 300],
        'XGBR__max_depth': [3, 4, 5, 6, 7],
        'XGBR__learning_rate': [0.01, 0.05, 0.1, 0.2],
        'XGBR__subsample': [0.6, 0.8, 1.0],
        'XGBR__colsample_bytree': [0.6, 0.8, 1.0],
        'XGBR__gamma': [0, 0.1, 0.2, 0.3],
        'XGBR__reg_alpha': [0, 0.01, 0.1],
        'XGBR__reg_lambda': [1, 1.5, 2],
    }


svrSearch=RandomizedSearchCV(estimator=svr_pipeline, param_distributions=svrparam_grid, cv=tscv, n_iter=5, n_jobs=-1,scoring="neg_mean_squared_error")

rfrSearch=RandomizedSearchCV(estimator=rfr_pipeline, param_distributions=rfrparam_grid, cv=tscv, n_iter=5, n_jobs=-1,scoring="neg_mean_squared_error")

gbrSearch=RandomizedSearchCV(estimator=gbr_pipeline, param_distributions=gbrparam_grid, cv=tscv, n_iter=5, n_jobs=-1,scoring="neg_mean_squared_error")

xgbSearch=RandomizedSearchCV(estimator=xgb_pipeline, param_distributions=xgbparam_grid, cv=tscv, n_iter=5, n_jobs=-1,scoring="neg_mean_squared_error")


models = {
    "SVR":svrSearch,
    "RFR":rfrSearch,
    "GBR":gbrSearch,
    "XGBR":xgbSearch
    }



In [ ]:
results=[]
for name,model in models.items():
  print(f"{name} is being tuned...\n")
  model.fit(X_train,Y_train)


SVR is being tuned...

RFR is being tuned...

GBR is being tuned...

XGBR is being tuned...



In [ ]:
voting=VotingRegressor(estimators=[("svr",svrSearch.best_estimator_),
 ("rfr",rfrSearch.best_estimator_),("xgbr",xgbSearch.best_estimator_)]) # Added the estimator name "xgbr" for xgbSearch.best_estimator_

stacking=StackingRegressor(estimators=[("rfr",rfrSearch.best_estimator_),("gbr",gbrSearch.best_estimator_),("xgbr",xgbSearch.best_estimator_)],final_estimator=SVR())

In [ ]:
stacking.fit(X_train,Y_train)

StackingRegressor(estimators=[('rfr',
                               Pipeline(steps=[('scaler', StandardScaler()),
                                               ('RFR',
                                                RandomForestRegressor(min_samples_leaf=3,
                                                                      random_state=42))])),
                              ('gbr',
                               Pipeline(steps=[('scaler', StandardScaler()),
                                               ('GBR',
                                                GradientBoostingRegressor(criterion='squared_error',
                                                                          learning_rate=0.05,
                                                                          max_depth=4,
                                                                          n_estimators=500,
                                                                          random_state=42,
                                                                          subsample=0.8))])),
                              ('xgbr'...
                                                             importance_type=None,
                                                             interaction_constraints=None,
                                                             learning_rate=0.05,
                                                             max_bin=None,
                                                             max_cat_threshold=None,
                                                             max_cat_to_onehot=None,
                                                             max_delta_step=None,
                                                             max_depth=3,
                                                             max_leaves=None,
                                                             min_child_weight=None,
                                                             missing=nan,
                                                             monotone_constraints=None,
                                                             multi_strategy=None,
                                                             n_estimators=200,
                                                             n_jobs=None,
                                                             num_parallel_tree=None,
                                                             random_state=42, ...))]))],
                  final_estimator=SVR())

In [ ]:
voting.fit(X_train,Y_train)

VotingRegressor(estimators=[('svr',
                             Pipeline(steps=[('scaler', StandardScaler()),
                                             ('SVR',
                                              SVR(C=5, epsilon=0.01,
                                                  gamma='auto'))])),
                            ('rfr',
                             Pipeline(steps=[('scaler', StandardScaler()),
                                             ('RFR',
                                              RandomForestRegressor(min_samples_leaf=3,
                                                                    random_state=42))])),
                            ('xgbr',
                             Pipeline(steps=[('scaler', StandardScaler()),
                                             ('XGBR',
                                              XGBRegressor(base_score=None,
                                                           booster=None,
                                                           cal...
                                                           grow_policy=None,
                                                           importance_type=None,
                                                           interaction_constraints=None,
                                                           learning_rate=0.05,
                                                           max_bin=None,
                                                           max_cat_threshold=None,
                                                           max_cat_to_onehot=None,
                                                           max_delta_step=None,
                                                           max_depth=3,
                                                           max_leaves=None,
                                                           min_child_weight=None,
                                                           missing=nan,
                                                           monotone_constraints=None,
                                                           multi_strategy=None,
                                                           n_estimators=200,
                                                           n_jobs=None,
                                                           num_parallel_tree=None,
                                                           random_state=42, ...))]))])

In [ ]:
models["Voting"]=voting
models["Stacking"]=stacking

In [ ]:
models

{'SVR': RandomizedSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
                    estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                              ('SVR', SVR())]),
                    n_iter=5, n_jobs=-1,
                    param_distributions={'SVR__C': [0.5, 1, 5],
                                         'SVR__epsilon': [0.01, 0.05, 0.1],
                                         'SVR__gamma': ['scale', 'auto'],
                                         'SVR__kernel': ['rbf']},
                    scoring='neg_mean_squared_error'),
 'RFR': RandomizedSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
                    estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                              ('RFR',
                                               RandomForestRegressor(random_state=42))]),
                    n_iter=5, n_jobs=-1,
                 

In [ ]:
results_dict = {}

for name, model in models.items():
    Y_preds = model.predict(X_test)

    # Print evaluation metrics
    print(f"{name}...")
    MAE = mean_absolute_error(Y_test, Y_preds)
    MSE = mean_squared_error(Y_test, Y_preds)
    RMSE = root_mean_squared_error(Y_test, Y_preds)
    MLE = mean_squared_log_error(Y_test, Y_preds)
    R2 = r2_score(Y_test, Y_preds)

    print(f"MEAN ABSOLUTE ERROR: {MAE}")
    print(f"MEAN SQUARED ERROR: {MSE}")
    print(f"ROOT MEAN SQUARED ERROR: {RMSE}")
    print(f"MEAN SQUARED LOG ERROR: {MLE}")
    print(f"R2 SCORE: {R2}")
    print("-------------------------------------------------------------------\n")

    results_dict[name] = Y_preds.tolist()  # Convert to list to ensure compatibility

# Create DataFrame using from_dict with orient='index'
results_df = pd.DataFrame.from_dict(results_dict, orient='index', columns=[f'Pred_{i}' for i in range(len(Y_preds))])


SVR...
MEAN ABSOLUTE ERROR: 0.37095962392606563
MEAN SQUARED ERROR: 1.5275076170499393
ROOT MEAN SQUARED ERROR: 1.2359237909555505
MEAN SQUARED LOG ERROR: 0.17587890651939628
R2 SCORE: 0.15124053725386422
-------------------------------------------------------------------

RFR...
MEAN ABSOLUTE ERROR: 0.15585269003224245
MEAN SQUARED ERROR: 0.24907302478196638
ROOT MEAN SQUARED ERROR: 0.4990721639021419
MEAN SQUARED LOG ERROR: 0.011052389775071756
R2 SCORE: 0.8616025973691853
-------------------------------------------------------------------

GBR...
MEAN ABSOLUTE ERROR: 0.15364800307842832
MEAN SQUARED ERROR: 0.24099496566888318
ROOT MEAN SQUARED ERROR: 0.49091238084701344
MEAN SQUARED LOG ERROR: 0.010616704128534332
R2 SCORE: 0.8660911701503107
-------------------------------------------------------------------

XGBR...
MEAN ABSOLUTE ERROR: 0.20813675224781036
MEAN SQUARED ERROR: 0.3807412385940552
ROOT MEAN SQUARED ERROR: 0.6170423030853271
MEAN SQUARED LOG ERROR: 0.01908273808658123

In [ ]:
results_df["Predicted_Values"] = results_df.values.tolist() # Convert DataFrame values to a list of lists
results_df = results_df.reset_index()
results_df = results_df[["index", "Predicted_Values"]]
results_df.columns = ["Model", "Predicted_Values"]

In [ ]:
results_df=results_df[results_df["Model"]!="Predicted_Values"]
results_df

,Model,Predicted_Values
0,SVR,"[-0.1933259891871683, 1.7275014835111846, 0.77..."
1,RFR,"[-0.19317074046221336, 1.715443356566833, 0.77..."
2,GBR,"[-0.1943618565268946, 1.732384542041446, 0.779..."
3,XGBR,"[-0.19482539594173431, 1.981188178062439, 0.77..."
4,Voting,"[-0.1937740418637053, 1.8080443393801522, 0.77..."
5,Stacking,"[-0.1954441160224559, 0.20513071526583315, 0.6..."


In [ ]:
Y_test

,Tomorrow
25890,-0.188863
25891,1.772179
25892,0.774136
25893,0.801320
25894,-0.279717
...,...
36975,1.023132
36976,-0.218026
36977,0.673976
36978,4.650157


In [ ]:

# Create line plots for each model with error annotations and outlier highlighting
for index, row in results_df.iterrows():
    model_name = row['Model']
    predicted = np.array(row['Predicted_Values'])
    actual = np.array(Y_test)

    # Compute absolute error
    errors = np.abs(actual - predicted)
    mae = mean_absolute_error(actual, predicted)

    # Identify outliers: errors greater than a threshold (e.g., 1.5 * MAE)
    threshold = 1.5 * mae
    outliers = np.where(errors > threshold)[0]

    plt.figure(figsize=(10, 4))
    plt.plot(actual, label='Actual', color='black', linewidth=2)
    plt.plot(predicted, label='Predicted', linestyle='--', color='red')

    # Highlight outliers
    plt.scatter(outliers, predicted[outliers], color='blue', label='High Error', zorder=5)

    # Annotate with MAE
    plt.title(f'{model_name} Predictions vs Actual\nMAE: {mae:.2f}', fontsize=13)
    plt.xlabel('Sample Index')
    plt.ylabel('Target Value')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.3)
    plt.tight_layout()
    plt.show()



In [ ]:
v_data=pd.read_csv("/content/drive/MyDrive/stocks/Extracted.csv",index_col=0)
v_data

In [ ]:
X_validate=v_data.drop(columns={"Date","Stock symbol","Tomorrow","Target"})
Y_validate=v_data[["Tomorrow"]]

In [ ]:
X_validate=X_validate[X_train.columns]
X_validate

In [ ]:
Y_validate

 On comparing , we could clearly see that Gradient Boosting, Random Forest Regressor and XGBRegression are the top 3 models for Stocks prediction

In [ ]:
valid_models = {
    "RFR":rfrSearch,
    "GBR":gbrSearch,
    "XGBR":xgbSearch
    }

In [ ]:
validation_results={}
for name,model in valid_models:
  print(f"{name}...")
  Y_valid_pred = model.predict(X_validate)
  Mae = mean_absolute_error(Y_validate, Y_valid_pred)
  Mse = mean_squared_error(Y_validate, Y_valid_pred)
  Rmse = root_mean_squared_error(Y_validate, Y_valid_pred)
  Mle = mean_squared_log_error(Y_validate, Y_valid_pred)
  r2 = r2_score(Y_validate, Y_valid_pred)
  print(f"MEAN ABSOLUTE ERROR: {Mae}")
  print(f"MEAN SQUARED ERROR: {Mse}")
  print(f"ROOT MEAN SQUARED ERROR: {Rmse}")
  print(f"MEAN SQUARED LOG ERROR: {Mle}")
  print(f"R2 SCORE: {r2}")
  print("-------------------------------------------------------------------\n")
  validation_results[name] = Y_valid_pred.tolist()  # Convert to list to ensure compatibility

# Create DataFrame using from_dict with orient='index'
validation_df = pd.DataFrame.from_dict(validation_results, orient='index', columns=[f'Pred_{i}' for i in range(len(Y_valid_pred))])
validation_df["Validation_Predicted_Values"] = validation_df.values.tolist() # Convert DataFrame values to a list of lists
validation_df = validation_df.reset_index()
validation_df = validation_df[["index", "Validation_Predicted_Values"]]
validation_df.columns = ["Model", "Validation_Predicted_Values"]